In [1]:
import json


file_path = "/content/Digestive_System_508-bbox.json"
with open(file_path, 'r') as file:
    data = json.load(file)

images = []
tables = []
captions = []

for item in data:
    if item['type'] == 'Image':
        images.append({
            'bbox': item.get('bbox'),
            'properties': item.get('properties'),
            'text_representation': item.get('text_representation')
        })
    elif item['type'] == 'Table':
        tables.append({
            'bbox': item.get('bbox'),
            'properties': item.get('properties'),
            'text_representation': item.get('text_representation')
        })
    elif item['type'] == 'Caption':
        captions.append({
            'bbox': item.get('bbox'),
            'properties': item.get('properties'),
            'text': item.get('text_representation')
        })


print("Images:", images)
print("Tables:", tables)
print("Captions:", captions)

Images: [{'bbox': [0.06785435844870175, 0.3083936101740057, 0.1670273365693934, 0.4612411776455966], 'properties': {'score': 0.35441550612449646, 'image_size': [189, 357], 'image_mode': 'RGB', 'image_format': None, 'page_number': 1}, 'text_representation': 'HTVH NIH National Institute of Diabetes and Digestive and Kidney Diseases'}, {'bbox': [0.0732449161305147, 0.3093439830433239, 0.15920937930836396, 0.37621152010830966], 'properties': {'score': 0.4814634919166565, 'image_size': [166, 167], 'image_mode': 'RGB', 'image_format': None, 'page_number': 1}, 'text_representation': ''}, {'bbox': [0.07908978630514706, 0.39323214444247157, 0.15881108901079963, 0.43187042236328127], 'properties': {'score': 0.48780766129493713, 'image_size': [156, 105], 'image_mode': 'RGB', 'image_format': None, 'page_number': 1}, 'text_representation': 'NIH'}, {'bbox': [0.6039058909696691, 0.30837296919389207, 0.9432667451746324, 0.6745396839488637], 'properties': {'score': 0.8949092626571655, 'image_size': [59

In [3]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 60.4 MB/s eta 0:00:00


In [4]:
pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 33.7 MB/s eta 0:00:00


In [6]:
import fitz  # PyMuPDF
import json
from PIL import Image
import io

# Paths to the files
json_path = "/content/Digestive_System_508-bbox.json"
pdf_path = "/content/Digestive_System_508.pdf"

# Load JSON data
with open(json_path, 'r') as f:
    json_data = json.load(f)

# Filter for images
images_data = [item for item in json_data if item["type"] == "Image"]

# Open the PDF
doc = fitz.open(pdf_path)

# Function to extract and display images
def extract_and_display_images(doc, images_data):
    for image in images_data:
        page_num = image["properties"]["page_number"] - 1  # Convert to 0-indexed
        bbox = image["bbox"]  # Bounding box

        # Open the page
        page = doc.load_page(page_num)

        # Convert bounding box coordinates to absolute
        rect = fitz.Rect(bbox[0] * page.rect.width, bbox[1] * page.rect.height,
                         bbox[2] * page.rect.width, bbox[3] * page.rect.height)

        # Extract the image
        pix = page.get_pixmap(clip=rect)
        img = Image.open(io.BytesIO(pix.tobytes("png")))

        # Display the image
        img.show()

# Extract and display images
extract_and_display_images(doc, images_data)

In [4]:
import json
from pdfplumber import open as open_pdf

# Paths to the files
json_path = "/content/Digestive_System_508-bbox.json"
pdf_path = "/content/Digestive_System_508.pdf"

# Load JSON data
with open(json_path, 'r') as f:
    json_data = json.load(f)

# Filter out Images, Tables, and Captions
content = {"images": [], "tables": [], "captions": []}
for item in json_data:
    if item["type"] in content:
        content[item["type"]].append({
            "bbox": item["bbox"],
            "page": item["properties"]["page_number"],
            "text": item.get("text_representation")
        })

# Extract visual data from the PDF
with open_pdf(pdf_path) as pdf:
    for page_number, page_data in enumerate(pdf.pages, start=1):
        if any(item["page"] == page_number for items in content.values() for item in items):
            print(f"Processing page {page_number}...")
            # Example: Extract images from this page
            images = page_data.images
            tables = page_data.extract_tables()

# Display the extracted JSON content for verification
print("Extracted content from JSON:")
print(json.dumps(content, indent=2))

Extracted content from JSON:
{
  "images": [],
  "tables": [],
  "captions": []
}


In [7]:
import fitz  # PyMuPDF
import json
from PIL import Image
import io
import os

# Prompt user for file paths
json_path = input("Enter the path to the JSON file: ")
pdf_path = input("Enter the path to the PDF file: ")
output_dir = input("Enter the directory to save extracted images: ")

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load JSON data
with open(json_path, 'r') as f:
    json_data = json.load(f)

# Filter for images
images_data = [item for item in json_data if item["type"] == "Image"]

# Open the PDF
doc = fitz.open(pdf_path)

# Function to extract and save images
def extract_and_save_images(doc, images_data, output_dir):
    for idx, image in enumerate(images_data):
        page_num = image["properties"]["page_number"] - 1  # Convert to 0-indexed
        bbox = image["bbox"]  # Bounding box

        # Open the page
        page = doc.load_page(page_num)

        # Convert bounding box coordinates to absolute
        rect = fitz.Rect(
            bbox[0] * page.rect.width, bbox[1] * page.rect.height,
            bbox[2] * page.rect.width, bbox[3] * page.rect.height
        )

        # Extract the image
        pix = page.get_pixmap(clip=rect)
        img = Image.open(io.BytesIO(pix.tobytes("png")))

        # Save the image
        output_path = os.path.join(output_dir, f"image_{idx + 1}.png")
        img.save(output_path)
        print(f"Image saved to {output_path}")

# Extract and save images
extract_and_save_images(doc, images_data, output_dir)

Enter the path to the JSON file: /content/Digestive_System_508-bbox.json
Enter the path to the PDF file: /content/Digestive_System_508.pdf
Enter the directory to save extracted images: extracted_images
Image saved to extracted_images/image_1.png
Image saved to extracted_images/image_2.png
Image saved to extracted_images/image_3.png
Image saved to extracted_images/image_4.png
Image saved to extracted_images/image_5.png
Image saved to extracted_images/image_6.png
Image saved to extracted_images/image_7.png
Image saved to extracted_images/image_8.png


In [8]:
import fitz  # PyMuPDF
import json
from PIL import Image
import io
import os

# File paths
json_path = "/content/Digestive_System_508-bbox.json"
output_dir = "extracted_content"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load JSON data
with open(json_path, 'r') as f:
    json_data = json.load(f)

# Categorize content from JSON
images_data = [item for item in json_data if item["type"] == "Image"]
tables_data = [item for item in json_data if item["type"] == "Table"]
captions_data = [item for item in json_data if item["type"] == "Caption"]

# Function to extract and save images
def extract_and_save_images(images_data, output_dir):
    for idx, image in enumerate(images_data):
        # Assuming image data contains raw image bytes or paths
        bbox = image.get("bbox")
        image_bytes = image.get("image_data")  # Placeholder for actual image data

        if image_bytes:
            img = Image.open(io.BytesIO(image_bytes))
            output_path = os.path.join(output_dir, f"image_{idx + 1}.png")
            img.save(output_path)
            print(f"Image saved to {output_path}")

# Function to extract and save tables
def extract_and_save_tables(tables_data, output_dir):
    tables_output_path = os.path.join(output_dir, "tables.json")
    with open(tables_output_path, "w") as f:
        json.dump(tables_data, f, indent=2)
    print(f"Tables saved to {tables_output_path}")

# Function to extract and save captions
def extract_and_save_captions(captions_data, output_dir):
    captions_output_path = os.path.join(output_dir, "captions.json")
    with open(captions_output_path, "w") as f:
        json.dump(captions_data, f, indent=2)
    print(f"Captions saved to {captions_output_path}")

# Extract and save content
extract_and_save_images(images_data, output_dir)
extract_and_save_tables(tables_data, output_dir)
extract_and_save_captions(captions_data, output_dir)

Tables saved to extracted_content/tables.json
Captions saved to extracted_content/captions.json


In [9]:
import fitz  # PyMuPDF
import json
from PIL import Image, ImageDraw, ImageFont
import io
import os

# File paths
json_path = "/content/Digestive_System_508-bbox.json"
pdf_path = "/content/Digestive_System_508.pdf"
output_dir = "extracted_content"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load JSON data
with open(json_path, 'r') as f:
    json_data = json.load(f)

# Categorize content from JSON
images_data = [item for item in json_data if item["type"] == "Image"]
tables_data = [item for item in json_data if item["type"] == "Table"]
captions_data = [item for item in json_data if item["type"] == "Caption"]

# Open the PDF
doc = fitz.open(pdf_path)

# Function to extract and save images with captions
def extract_and_save_images_with_captions(doc, images_data, captions_data, output_dir):
    for idx, image in enumerate(images_data):
        page_num = image["properties"]["page_number"] - 1  # Convert to 0-indexed
        bbox = image["bbox"]  # Bounding box

        # Open the page
        page = doc.load_page(page_num)

        # Convert bounding box coordinates to absolute
        rect = fitz.Rect(
            bbox[0] * page.rect.width, bbox[1] * page.rect.height,
            bbox[2] * page.rect.width, bbox[3] * page.rect.height
        )

        # Extract the image
        pix = page.get_pixmap(clip=rect)
        img = Image.open(io.BytesIO(pix.tobytes("png")))

        # Find the closest caption
        caption_text = ""
        for caption in captions_data:
            if caption["properties"]["page_number"] == page_num + 1:
                caption_bbox = caption["bbox"]
                caption_rect = fitz.Rect(
                    caption_bbox[0] * page.rect.width, caption_bbox[1] * page.rect.height,
                    caption_bbox[2] * page.rect.width, caption_bbox[3] * page.rect.height
                )
                if rect.intersects(caption_rect):
                    caption_text = caption["text_representation"]
                    break

        # Add caption to the image
        if caption_text:
            draw = ImageDraw.Draw(img)
            font = ImageFont.load_default()
            text_position = (10, img.height - 20)  # Add caption at the bottom
            draw.text(text_position, caption_text, fill="black", font=font)

        # Save the image
        output_path = os.path.join(output_dir, f"image_with_caption_{idx + 1}.png")
        img.save(output_path)
        print(f"Image with caption saved to {output_path}")

# Function to extract and save tables with captions
def extract_and_save_tables_with_captions(doc, tables_data, captions_data, output_dir):
    for idx, table in enumerate(tables_data):
        page_num = table["properties"]["page_number"] - 1  # Convert to 0-indexed
        bbox = table["bbox"]  # Bounding box

        # Open the page
        page = doc.load_page(page_num)

        # Convert bounding box coordinates to absolute
        rect = fitz.Rect(
            bbox[0] * page.rect.width, bbox[1] * page.rect.height,
            bbox[2] * page.rect.width, bbox[3] * page.rect.height
        )

        # Extract the table as an image
        pix = page.get_pixmap(clip=rect)
        img = Image.open(io.BytesIO(pix.tobytes("png")))

        # Find the closest caption
        caption_text = ""
        for caption in captions_data:
            if caption["properties"]["page_number"] == page_num + 1:
                caption_bbox = caption["bbox"]
                caption_rect = fitz.Rect(
                    caption_bbox[0] * page.rect.width, caption_bbox[1] * page.rect.height,
                    caption_bbox[2] * page.rect.width, caption_bbox[3] * page.rect.height
                )
                if rect.intersects(caption_rect):
                    caption_text = caption["text_representation"]
                    break

        # Add caption to the table image
        if caption_text:
            draw = ImageDraw.Draw(img)
            font = ImageFont.load_default()
            text_position = (10, img.height - 20)  # Add caption at the bottom
            draw.text(text_position, caption_text, fill="black", font=font)

        # Save the table image
        output_path = os.path.join(output_dir, f"table_with_caption_{idx + 1}.png")
        img.save(output_path)
        print(f"Table with caption saved to {output_path}")

# Extract and save images and tables with captions
extract_and_save_images_with_captions(doc, images_data, captions_data, output_dir)
extract_and_save_tables_with_captions(doc, tables_data, captions_data, output_dir)

Image with caption saved to extracted_content/image_with_caption_1.png
Image with caption saved to extracted_content/image_with_caption_2.png
Image with caption saved to extracted_content/image_with_caption_3.png
Image with caption saved to extracted_content/image_with_caption_4.png
Image with caption saved to extracted_content/image_with_caption_5.png
Image with caption saved to extracted_content/image_with_caption_6.png
Image with caption saved to extracted_content/image_with_caption_7.png
Image with caption saved to extracted_content/image_with_caption_8.png


In [13]:
import json
from PIL import Image, ImageDraw, ImageFont
import os

json_path = "/content/Digestive_System_508-bbox.json"
output_dir = "extracted_visualizations"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load JSON data
with open(json_path, 'r') as f:
    json_data = json.load(f)

# Categorize content from JSON
images_data = [item for item in json_data if item["type"] == "Image"]
tables_data = [item for item in json_data if item["type"] == "Table"]
captions_data = [item for item in json_data if item["type"] == "Caption"]

# Function to recreate and save visual representations
def recreate_visual_representation(images_data, tables_data, captions_data, output_dir):
    for idx, item in enumerate(images_data + tables_data):
        page_width, page_height = 800, 1000  # Assume a fixed page size for visualization
        bbox = item["bbox"]  # Bounding box
        item_type = item["type"]  # "Image" or "Table"

        # Create a blank canvas for the page
        canvas = Image.new("RGB", (page_width, page_height), "white")
        draw = ImageDraw.Draw(canvas)

        # Draw the bounding box
        rect_coords = (
            bbox[0] * page_width,
            bbox[1] * page_height,
            bbox[2] * page_width,
            bbox[3] * page_height,
        )
        draw.rectangle(rect_coords, outline="black", width=3)
        draw.text((rect_coords[0] + 5, rect_coords[1] - 15), item_type, fill="black")

        # Find the closest caption
        caption_text = ""
        for caption in captions_data:
            caption_bbox = caption["bbox"]
            if (
                rect_coords[0] <= caption_bbox[2] * page_width
                and rect_coords[2] >= caption_bbox[0] * page_width
                and rect_coords[1] <= caption_bbox[3] * page_height
                and rect_coords[3] >= caption_bbox[1] * page_height
            ):
                caption_text = caption["text_representation"]
                break

        # Add the caption to the canvas
        if caption_text:
            font = ImageFont.load_default()
            caption_position = (rect_coords[0], rect_coords[3] + 10)
            draw.text(caption_position, caption_text, fill="black", font=font)

        # Save the visualization
        output_path = os.path.join(output_dir, f"{item_type.lower()}_{idx + 1}.png")
        canvas.save(output_path)
        print(f"{item_type} visualization saved to {output_path}")

# Recreate and save visual representations
recreate_visual_representation(images_data, tables_data, captions_data, output_dir)

Image visualization saved to extracted_visualizations/image_1.png
Image visualization saved to extracted_visualizations/image_2.png
Image visualization saved to extracted_visualizations/image_3.png
Image visualization saved to extracted_visualizations/image_4.png
Image visualization saved to extracted_visualizations/image_5.png
Image visualization saved to extracted_visualizations/image_6.png
Image visualization saved to extracted_visualizations/image_7.png
Image visualization saved to extracted_visualizations/image_8.png


In [18]:
import json
from PIL import Image, ImageDraw, ImageFont
import io
import os
import base64

# File paths
json_path = "/content/Digestive_System_508-bbox.json"
output_dir = "extracted_co"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load JSON data
with open(json_path, 'r') as f:
    json_data = json.load(f)

# Categorize content from JSON
images_data = [item for item in json_data if item["type"] == "Image"]
tables_data = [item for item in json_data if item["type"] == "Table"]
captions_data = [item for item in json_data if item["type"] == "Caption"]

# Debugging: Print counts
print(f"Found {len(images_data)} images, {len(tables_data)} tables, and {len(captions_data)} captions in the JSON.")

# Function to decode and save images with captions
def extract_and_save_images_with_captions(images_data, captions_data, output_dir):
    for idx, image in enumerate(images_data):
        print(f"Processing image {idx + 1}/{len(images_data)}...")
        bbox = image["bbox"]  # Bounding box
        image_data_base64 = image.get("image_data")  # Base64-encoded image data

        if image_data_base64:
            try:
                # Decode the image data
                image_bytes = base64.b64decode(image_data_base64)
                img = Image.open(io.BytesIO(image_bytes))

                # Debugging: Log image size
                print(f"Image {idx + 1} loaded: size={img.size}")

                # Find the closest caption
                caption_text = ""
                for caption in captions_data:
                    caption_bbox = caption["bbox"]
                    # Check if the caption overlaps with the image bbox
                    if (bbox[0] <= caption_bbox[2] and bbox[2] >= caption_bbox[0] and
                            bbox[1] <= caption_bbox[3] and bbox[3] >= caption_bbox[1]):
                        caption_text = caption["text_representation"]
                        break

                # Add caption to the image
                if caption_text:
                    draw = ImageDraw.Draw(img)
                    font = ImageFont.load_default()
                    text_position = (10, img.height - 20)  # Add caption at the bottom
                    draw.text(text_position, caption_text, fill="black", font=font)

                # Save the image
                output_path = os.path.join(output_dir, f"image_with_caption_{idx + 1}.png")
                img.save(output_path)
                print(f"Image with caption saved to {output_path}")
            except Exception as e:
                print(f"Error processing image {idx + 1}: {e}")
        else:
            print(f"No image data found for image {idx + 1}.")

# Function to decode and save tables with captions
def extract_and_save_tables_with_captions(tables_data, captions_data, output_dir):
    for idx, table in enumerate(tables_data):
        print(f"Processing table {idx + 1}/{len(tables_data)}...")
        bbox = table["bbox"]  # Bounding box
        table_data_base64 = table.get("table_data")  # Base64-encoded table image data

        if table_data_base64:
            try:
                # Decode the table data
                table_bytes = base64.b64decode(table_data_base64)
                img = Image.open(io.BytesIO(table_bytes))

                # Debugging: Log table size
                print(f"Table {idx + 1} loaded: size={img.size}")

                # Find the closest caption
                caption_text = ""
                for caption in captions_data:
                    caption_bbox = caption["bbox"]
                    # Check if the caption overlaps with the table bbox
                    if (bbox[0] <= caption_bbox[2] and bbox[2] >= caption_bbox[0] and
                            bbox[1] <= caption_bbox[3] and bbox[3] >= caption_bbox[1]):
                        caption_text = caption["text_representation"]
                        break

                # Add caption to the table image
                if caption_text:
                    draw = ImageDraw.Draw(img)
                    font = ImageFont.load_default()
                    text_position = (10, img.height - 20)  # Add caption at the bottom
                    draw.text(text_position, caption_text, fill="black", font=font)

                # Save the table image
                output_path = os.path.join(output_dir, f"table_with_caption_{idx + 1}.png")
                img.save(output_path)
                print(f"Table with caption saved to {output_path}")
            except Exception as e:
                print(f"Error processing table {idx + 1}: {e}")
        else:
            print(f"No table data found for table {idx + 1}.")

# Extract and save images and tables with captions
extract_and_save_images_with_captions(images_data, captions_data, output_dir)
extract_and_save_tables_with_captions(tables_data, captions_data, output_dir)


Found 8 images, 0 tables, and 4 captions in the JSON.
Processing image 1/8...
No image data found for image 1.
Processing image 2/8...
No image data found for image 2.
Processing image 3/8...
No image data found for image 3.
Processing image 4/8...
No image data found for image 4.
Processing image 5/8...
No image data found for image 5.
Processing image 6/8...
No image data found for image 6.
Processing image 7/8...
No image data found for image 7.
Processing image 8/8...
No image data found for image 8.


In [19]:
import json
from PIL import Image, ImageDraw, ImageFont
import io
import os
import base64

# File paths
json_path = "/content/Digestive_System_508-bbox.json"
output_dir = "retrieved_content"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load JSON data
with open(json_path, 'r') as f:
    json_data = json.load(f)

# Categorize content from JSON
images_data = [item for item in json_data if item["type"] == "Image"]
tables_data = [item for item in json_data if item["type"] == "Table"]

# Function to decode and save images
def decode_and_save_images(images_data, output_dir):
    for idx, image in enumerate(images_data):
        binary_data = image.get("binary_data")  # Assuming 'binary_data' holds the image in binary
        if binary_data:
            try:
                # Convert binary to bytes
                image_bytes = base64.b64decode(binary_data)
                img = Image.open(io.BytesIO(image_bytes))

                # Save the image
                output_path = os.path.join(output_dir, f"image_{idx + 1}.png")
                img.save(output_path)
                print(f"Image saved to {output_path}")
            except Exception as e:
                print(f"Failed to decode and save image {idx + 1}: {e}")
        else:
            print(f"No binary data found for image {idx + 1}.")

# Function to decode and save tables
def decode_and_save_tables(tables_data, output_dir):
    for idx, table in enumerate(tables_data):
        binary_data = table.get("binary_data")  # Assuming 'binary_data' holds the table in binary
        if binary_data:
            try:
                # Convert binary to bytes
                table_bytes = base64.b64decode(binary_data)
                img = Image.open(io.BytesIO(table_bytes))

                # Save the table image
                output_path = os.path.join(output_dir, f"table_{idx + 1}.png")
                img.save(output_path)
                print(f"Table saved to {output_path}")
            except Exception as e:
                print(f"Failed to decode and save table {idx + 1}: {e}")
        else:
            print(f"No binary data found for table {idx + 1}.")

# Decode and save images and tables
decode_and_save_images(images_data, output_dir)
decode_and_save_tables(tables_data, output_dir)

No binary data found for image 1.
No binary data found for image 2.
No binary data found for image 3.
No binary data found for image 4.
No binary data found for image 5.
No binary data found for image 6.
No binary data found for image 7.
No binary data found for image 8.


In [20]:
import json
from PIL import Image, ImageDraw, ImageFont
import os

# File paths
json_path = "/content/Digestive_System_508-bbox.json"
output_dir = "reconstructed_layouts"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load JSON data
with open(json_path, 'r') as f:
    json_data = json.load(f)

# Categorize content from JSON
images_data = [item for item in json_data if item["type"] == "Image"]
tables_data = [item for item in json_data if item["type"] == "Table"]
captions_data = [item for item in json_data if item["type"] == "Caption"]

# Function to reconstruct layout from JSON metadata
def reconstruct_layout(images_data, tables_data, captions_data, output_dir):
    page_width, page_height = 800, 1000  # Assumed page size
    canvas = Image.new("RGB", (page_width, page_height), "white")
    draw = ImageDraw.Draw(canvas)

    # Draw images
    for image in images_data:
        bbox = image["bbox"]
        rect_coords = (
            bbox[0] * page_width,
            bbox[1] * page_height,
            bbox[2] * page_width,
            bbox[3] * page_height,
        )
        draw.rectangle(rect_coords, outline="blue", width=2)
        draw.text((rect_coords[0], rect_coords[1] - 10), "Image", fill="blue")

    # Draw tables
    for table in tables_data:
        bbox = table["bbox"]
        rect_coords = (
            bbox[0] * page_width,
            bbox[1] * page_height,
            bbox[2] * page_width,
            bbox[3] * page_height,
        )
        draw.rectangle(rect_coords, outline="green", width=2)
        draw.text((rect_coords[0], rect_coords[1] - 10), "Table", fill="green")

    # Draw captions
    for caption in captions_data:
        bbox = caption["bbox"]
        text = caption["text_representation"]
        rect_coords = (
            bbox[0] * page_width,
            bbox[1] * page_height,
            bbox[2] * page_width,
            bbox[3] * page_height,
        )
        draw.rectangle(rect_coords, outline="red", width=1)
        draw.text((rect_coords[0], rect_coords[1] - 10), text, fill="red")

    # Save reconstructed page
    output_path = os.path.join(output_dir, "reconstructed_page.png")
    canvas.save(output_path)
    print(f"Reconstructed layout saved to {output_path}")

# Reconstruct and save the layout
reconstruct_layout(images_data, tables_data, captions_data, output_dir)

Reconstructed layout saved to reconstructed_layouts/reconstructed_page.png


In [6]:
import json
import base64
from PIL import Image
import pandas as pd
import io
import os

# File paths
json_path = "/content/Digestive_System_508-bbox.json"
output_dir = "extracted_data"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load JSON data
with open(json_path, "r") as f:
    json_data = json.load(f)

# Extract Images
def extract_images(json_data, output_dir):
    images = [item for item in json_data if item["type"] == "Image"]
    for idx, image in enumerate(images):
        binary_data = image.get("binary_representation")
        if binary_data:
            try:
                # Decode Base64 and save the image
                img_data = base64.b64decode(binary_data)
                image_size = image["properties"]["image_size"]
                image_mode = image["properties"]["image_mode"]
                img = Image.frombytes(image_mode, tuple(image_size), img_data)
                img.save(os.path.join(output_dir, f"image_{idx + 1}.png"))
                print(f"Image {idx + 1} saved successfully!")
            except Exception as e:
                print(f"Failed to decode image {idx + 1}: {e}")
        else:
            print(f"No binary data for image {idx + 1}.")

# Extract Tables
def extract_tables(json_data, output_dir):
    tables = [item for item in json_data if item["type"] == "table"]
    for idx, table in enumerate(tables):
        try:
            cells = table["table"]["cells"]
            rows = {}
            for cell in cells:
                row_idx = cell["rows"][0]
                col_idx = cell["cols"][0]
                content = cell.get("content", "")
                if row_idx not in rows:
                    rows[row_idx] = {}
                rows[row_idx][col_idx] = content

            # Create a pandas DataFrame
            df = pd.DataFrame.from_dict(rows, orient="index").sort_index(axis=1)
            df.to_csv(os.path.join(output_dir, f"table_{idx + 1}.csv"), index=False)
            print(f"Table {idx + 1} saved successfully!")
        except Exception as e:
            print(f"Failed to process table {idx + 1}: {e}")

# Run extraction functions
extract_images(json_data, output_dir)
extract_tables(json_data, output_dir)

Image 1 saved successfully!
Image 2 saved successfully!
Image 3 saved successfully!
Image 4 saved successfully!
Image 5 saved successfully!
Image 6 saved successfully!
Image 7 saved successfully!
Image 8 saved successfully!
Table 1 saved successfully!


In [23]:
import json
import base64
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import io
import os

# File paths
json_path = "/content/Digestive_System_508-bbox.json"
output_dir = "extracted_data1"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load JSON data
with open(json_path, "r") as f:
    json_data = json.load(f)

# Extract Images
def extract_images_with_captions(json_data, output_dir):
    images = [item for item in json_data if item["type"] == "Image"]
    captions = [item for item in json_data if item["type"] == "Caption"]

    for idx, image in enumerate(images):
        binary_data = image.get("binary_representation")
        if binary_data:
            try:
                # Decode Base64 and save the image
                img_data = base64.b64decode(binary_data)
                image_size = image["properties"]["image_size"]
                image_mode = image["properties"]["image_mode"]
                img = Image.frombytes(image_mode, tuple(image_size), img_data)

                # Find associated caption
                caption_text = ""
                image_bbox = image["bbox"]
                for caption in captions:
                    caption_bbox = caption["bbox"]
                    if (
                        image_bbox[0] <= caption_bbox[2]
                        and image_bbox[2] >= caption_bbox[0]
                        and image_bbox[1] <= caption_bbox[3]
                        and image_bbox[3] >= caption_bbox[1]
                    ):
                        caption_text = caption["text_representation"]
                        break

                # Add caption to the image
                if caption_text:
                    draw = ImageDraw.Draw(img)
                    font = ImageFont.load_default()
                    draw.text((10, img.height - 20), caption_text, fill="black", font=font)

                # Save the image
                img.save(os.path.join(output_dir, f"image_with_caption_{idx + 1}.png"))
                print(f"Image {idx + 1} with caption saved successfully!")
            except Exception as e:
                print(f"Failed to decode image {idx + 1}: {e}")
        else:
            print(f"No binary data for image {idx + 1}.")

# Extract Tables
def extract_tables_with_captions(json_data, output_dir):
    tables = [item for item in json_data if item["type"] == "table"]
    captions = [item for item in json_data if item["type"] == "Caption"]

    for idx, table in enumerate(tables):
        try:
            cells = table["table"]["cells"]
            rows = {}
            for cell in cells:
                row_idx = cell["rows"][0]
                col_idx = cell["cols"][0]
                content = cell.get("content", "")
                if row_idx not in rows:
                    rows[row_idx] = {}
                rows[row_idx][col_idx] = content

            # Create a pandas DataFrame
            df = pd.DataFrame.from_dict(rows, orient="index").sort_index(axis=1)

            # Find associated caption
            caption_text = ""
            table_bbox = table["bbox"]
            for caption in captions:
                caption_bbox = caption["bbox"]
                if (
                    table_bbox[0] <= caption_bbox[2]
                    and table_bbox[2] >= caption_bbox[0]
                    and table_bbox[1] <= caption_bbox[3]
                    and table_bbox[3] >= caption_bbox[1]
                ):
                    caption_text = caption["text_representation"]
                    break

            # Save the table and caption
            table_path = os.path.join(output_dir, f"table_with_caption_{idx + 1}.csv")
            df.to_csv(table_path, index=False)
            if caption_text:
                with open(table_path.replace(".csv", ".txt"), "w") as caption_file:
                    caption_file.write(caption_text)
            print(f"Table {idx + 1} with caption saved successfully!")
        except Exception as e:
            print(f"Failed to process table {idx + 1}: {e}")

# Run extraction functions
extract_images_with_captions(json_data, output_dir)
extract_tables_with_captions(json_data, output_dir)

Image 1 with caption saved successfully!
Image 2 with caption saved successfully!
Image 3 with caption saved successfully!
Image 4 with caption saved successfully!
Image 5 with caption saved successfully!
Image 6 with caption saved successfully!
Image 7 with caption saved successfully!
Image 8 with caption saved successfully!
Table 1 with caption saved successfully!


In [7]:
import json
import base64
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import io
import os

# File paths
json_path = "/content/Digestive_System_508-bbox.json"
output_dir = "data"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load JSON data
with open(json_path, "r") as f:
    json_data = json.load(f)

# Extract Images
def extract_images_with_captions(json_data, output_dir):
    images = [item for item in json_data if item["type"] == "Image"]
    captions = [item for item in json_data if item["type"] == "Caption"]

    for idx, image in enumerate(images):
        binary_data = image.get("binary_representation")
        if binary_data:
            try:
                # Decode Base64 and save the image
                img_data = base64.b64decode(binary_data)
                image_size = image["properties"]["image_size"]
                image_mode = image["properties"]["image_mode"]
                img = Image.frombytes(image_mode, tuple(image_size), img_data)

                # Find associated caption
                caption_text = ""
                image_bbox = image["bbox"]
                for caption in captions:
                    caption_bbox = caption["bbox"]
                    if (
                        caption_bbox[0] >= image_bbox[0]
                        and caption_bbox[2] <= image_bbox[2]
                        and caption_bbox[1] >= image_bbox[3]
                    ):
                        caption_text = caption["text_representation"]
                        break

                # Add caption to the image
                if caption_text:
                    new_height = img.height + 40
                    new_img = Image.new("RGB", (img.width, new_height), "white")
                    new_img.paste(img, (0, 0))
                    draw = ImageDraw.Draw(new_img)
                    font = ImageFont.load_default()
                    draw.text((10, img.height + 10), caption_text, fill="black", font=font)
                    img = new_img

                # Save the image
                img.save(os.path.join(output_dir, f"image_with_caption_{idx + 1}.png"))
                print(f"Image {idx + 1} with caption saved successfully!")
            except Exception as e:
                print(f"Failed to decode image {idx + 1}: {e}")
        else:
            print(f"No binary data for image {idx + 1}.")

# Extract Tables
def extract_tables_with_captions(json_data, output_dir):
    tables = [item for item in json_data if item["type"] == "table"]
    captions = [item for item in json_data if item["type"] == "Caption"]

    for idx, table in enumerate(tables):
        try:
            cells = table["table"]["cells"]
            rows = {}
            for cell in cells:
                row_idx = cell["rows"][0]
                col_idx = cell["cols"][0]
                content = cell.get("content", "")
                if row_idx not in rows:
                    rows[row_idx] = {}
                rows[row_idx][col_idx] = content

            # Create a pandas DataFrame
            df = pd.DataFrame.from_dict(rows, orient="index").sort_index(axis=1)

            # Find associated caption
            caption_text = ""
            table_bbox = table["bbox"]
            for caption in captions:
                caption_bbox = caption["bbox"]
                if (
                    caption_bbox[0] >= table_bbox[0]
                    and caption_bbox[2] <= table_bbox[2]
                    and caption_bbox[1] >= table_bbox[3]
                ):
                    caption_text = caption["text_representation"]
                    break

            # Save the table and caption
            table_path = os.path.join(output_dir, f"table_with_caption_{idx + 1}.csv")
            df.to_csv(table_path, index=False)
            if caption_text:
                with open(table_path.replace(".csv", ".txt"), "w") as caption_file:
                    caption_file.write(caption_text)
            print(f"Table {idx + 1} with caption saved successfully!")
        except Exception as e:
            print(f"Failed to process table {idx + 1}: {e}")

# Run extraction functions
extract_images_with_captions(json_data, output_dir)
extract_tables_with_captions(json_data, output_dir)

Image 1 with caption saved successfully!
Image 2 with caption saved successfully!
Image 3 with caption saved successfully!
Image 4 with caption saved successfully!
Image 5 with caption saved successfully!
Image 6 with caption saved successfully!
Image 7 with caption saved successfully!
Image 8 with caption saved successfully!
Table 1 with caption saved successfully!


In [1]:
pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.0 MB/s eta 0:00:00


In [39]:
from pinecone import Pinecone

# Pinecone configuration
PINECONE_API_KEY = "pcsk_4P1CDJ_3Dx22sxGN8Vq2hS7TXe2MWgD2gp1QFe4z8Qnj4g86rcK3HSA9xiPXEjxpTXrLrt"
PINECONE_INDEX_NAME = "demo"

# Output directory
output_dir = "pinecone_extracted_data"
os.makedirs(output_dir, exist_ok=True)

# Initialize the Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Ensure the index exists
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    print(f"Index '{PINECONE_INDEX_NAME}' does not exist. Please create it.")
    exit()

# Create an object for the Pinecone index
index = pc.Index(name=PINECONE_INDEX_NAME)

# Function to fetch and process data from Pinecone
def fetch_and_process_data_from_pinecone(index, output_dir):
    # Query Pinecone for all items (e.g., retrieve all vectors)
    result = index.query(vector=[0] * 1536, top_k=1000, include_metadata=True)  # Replace [0] * 1536 with a real query vector

    for item in result["matches"]:
        metadata = item["metadata"]

        # Process Images
        if metadata.get("type") == "Image":
            binary_data = metadata.get("binary_representation")
            if binary_data:
                try:
                    img_data = base64.b64decode(binary_data)
                    image_size = metadata.get("image_size", (800, 600))
                    image_mode = metadata.get("image_mode", "RGB")
                    img = Image.frombytes(image_mode, tuple(image_size), img_data)

                    # Add caption if available
                    caption_text = metadata.get("caption", "")
                    if caption_text:
                        new_height = img.height + 40
                        new_img = Image.new("RGB", (img.width, new_height), "white")
                        new_img.paste(img, (0, 0))
                        draw = ImageDraw.Draw(new_img)
                        font = ImageFont.load_default()
                        draw.text((10, img.height + 10), caption_text, fill="black", font=font)
                        img = new_img

                    # Save the image
                    img.save(os.path.join(output_dir, f"image_with_caption_{item['id']}.png"))
                    print(f"Image {item['id']} with caption saved successfully!")
                except Exception as e:
                    print(f"Failed to decode image {item['id']}: {e}")

        # Process Tables
        elif metadata.get("type") == "table":
            try:
                cells = metadata.get("table_cells", [])
                rows = {}
                for cell in cells:
                    row_idx = cell["rows"][0]
                    col_idx = cell["cols"][0]
                    content = cell.get("content", "")
                    if row_idx not in rows:
                        rows[row_idx] = {}
                    rows[row_idx][col_idx] = content

                # Create a pandas DataFrame
                df = pd.DataFrame.from_dict(rows, orient="index").sort_index(axis=1)

                # Save table and caption
                table_path = os.path.join(output_dir, f"table_with_caption_{item['id']}.csv")
                df.to_csv(table_path, index=False)

                caption_text = metadata.get("caption", "")
                if caption_text:
                    with open(table_path.replace(".csv", ".txt"), "w") as caption_file:
                        caption_file.write(caption_text)
                print(f"Table {item['id']} with caption saved successfully!")
            except Exception as e:
                print(f"Failed to process table {item['id']}: {e}")

# Fetch and process data
fetch_and_process_data_from_pinecone(index, output_dir)

Table aryn:f-dwz10usa1d4lwk43lwp57ez#aryn:c-q8t3unczu0smpu8yo60wd7w with caption saved successfully!
Table aryn:f-5m3jn27uji6pqemwkhm7cva#aryn:c-bu22r9o3vu5zgea20ameuzu with caption saved successfully!


In [44]:
from pinecone import Pinecone
import base64
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import os

# Pinecone configuration
PINECONE_API_KEY = "pcsk_4P1CDJ_3Dx22sxGN8Vq2hS7TXe2MWgD2gp1QFe4z8Qnj4g86rcK3HSA9xiPXEjxpTXrLrt"
PINECONE_INDEX_NAME = "demo"

# Output directory
output_dir = "pinecone_extracted_data1"
os.makedirs(output_dir, exist_ok=True)

# Initialize the Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Check if the index exists
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    print(f"Index '{PINECONE_INDEX_NAME}' does not exist. Please create it.")
    exit()

# Create an object for the Pinecone index
index = pc.Index(name=PINECONE_INDEX_NAME)

# Query Pinecone and debug results
result = index.query(vector=[0] * 1536, top_k=10, include_metadata=True)
print("Query Results:", result)

for item in result.get("matches", []):
    metadata = item["metadata"]
    print("Metadata:", metadata)

    # Process Images
    if metadata.get("type") == "Image":
        binary_data = metadata.get("binary_representation")
        if binary_data:
            try:
                print(f"Processing image: {item['id']}")
                img_data = base64.b64decode(binary_data)
                img = Image.open(io.BytesIO(img_data))

                # Add caption if available
                caption_text = metadata.get("caption", "")
                if caption_text:
                    print(f"Adding caption: {caption_text}")
                    new_height = img.height + 40
                    new_img = Image.new("RGB", (img.width, new_height), "white")
                    new_img.paste(img, (0, 0))
                    draw = ImageDraw.Draw(new_img)
                    font = ImageFont.load_default()
                    draw.text((10, img.height + 10), caption_text, fill="black", font=font)
                    img = new_img

                # Save the image
                img.save(os.path.join(output_dir, f"image_with_caption_{item['id']}.png"))
                print(f"Image {item['id']} saved successfully!")
            except Exception as e:
                print(f"Failed to decode image {item['id']}: {e}")

    # Process Tables
    elif metadata.get("type") == "table":
        try:
            print(f"Processing table: {item['id']}")
            cells = metadata.get("table_cells", [])
            rows = {}
            for cell in cells:
                row_idx = cell["rows"][0]
                col_idx = cell["cols"][0]
                content = cell.get("content", "")
                if row_idx not in rows:
                    rows[row_idx] = {}
                rows[row_idx][col_idx] = content

            # Create a pandas DataFrame
            df = pd.DataFrame.from_dict(rows, orient="index").sort_index(axis=1)

            # Save table and caption
            table_path = os.path.join(output_dir, f"table_with_caption_{item['id']}.csv")
            df.to_csv(table_path, index=False)

            caption_text = metadata.get("caption", "")
            if caption_text:
                print(f"Adding table caption: {caption_text}")
                with open(table_path.replace(".csv", ".txt"), "w") as caption_file:
                    caption_file.write(caption_text)
            print(f"Table {item['id']} saved successfully!")
        except Exception as e:
            print(f"Failed to process table {item['id']}: {e}")

Query Results: {'matches': [{'id': 'aryn:f-5m3jn27uji6pqemwkhm7cva#aryn:c-dyo0p75j2yuug33k0bmu87e',
              'metadata': {'bbox.x1': 0.24234186509076286,
                           'bbox.x2': 0.6996798885569853,
                           'bbox.y1': 0.2694771506569602,
                           'bbox.y2': 0.2853633533824574,
                           'properties._element_index': 1.0,
                           'properties.page_number': 1.0,
                           'properties.path': '/content/Digestive_System_508.pdf',
                           'properties.score': 0.5101474523544312,
                           'text_representation': 'National Digestive Diseases '
                                                  'Information Clearinghouse',
                           'type': 'Caption'},
              'score': 0.0,
              'values': []},
             {'id': 'aryn:f-dwz10usa1d4lwk43lwp57ez#aryn:c-wqig05tcr7tvkba0rylxtm1',
              'metadata': {'bbox.x1': 0.079089786

In [2]:
from pinecone import Pinecone
import base64
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import os

# Pinecone configuration
PINECONE_API_KEY = "pcsk_4P1CDJ_3Dx22sxGN8Vq2hS7TXe2MWgD2gp1QFe4z8Qnj4g86rcK3HSA9xiPXEjxpTXrLrt"
PINECONE_ENVIRONMENT = "us-east-1"
PINECONE_INDEX_NAME = "demo"

# Output directory
output_dir = "data12"
os.makedirs(output_dir, exist_ok=True)

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Ensure the index exists
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    print(f"Index '{PINECONE_INDEX_NAME}' does not exist. Creating it...")
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=1536,  # Replace with your vector dimension
        metric="cosine",  # Adjust metric to your requirements
    )
    print(f"Index '{PINECONE_INDEX_NAME}' created successfully!")

# Connect to the index
index = pc.Index(name=PINECONE_INDEX_NAME)

# Function to fetch and process Pinecone data
def fetch_and_process_pinecone_data(index, output_dir):
    # Query Pinecone for all items
    result = index.query(vector=[0] * 1536, top_k=1000, include_metadata=True)
    print("Fetched data from Pinecone.")

    # Separate images, tables, and captions
    images = [item for item in result["matches"] if item["metadata"].get("type") == "Image"]
    tables = [item for item in result["matches"] if item["metadata"].get("type") == "Table"]
    captions = [item for item in result["matches"] if item["metadata"].get("type") == "Caption"]

    # Process Images
    for idx, image in enumerate(images):
        metadata = image["metadata"]
        binary_data = metadata.get("binary_representation")
        if binary_data:
            try:
                img_data = base64.b64decode(binary_data)
                image_mode = metadata.get("properties.image_mode", "RGB")
                image_size = (int(metadata["properties.image_size.0"]), int(metadata["properties.image_size.1"]))
                img = Image.frombytes(image_mode, image_size, img_data)

                # Find associated caption
                caption_text = ""
                image_bbox = (
                    metadata["bbox.x1"],
                    metadata["bbox.y1"],
                    metadata["bbox.x2"],
                    metadata["bbox.y2"],
                )
                for caption in captions:
                    caption_bbox = (
                        caption["metadata"]["bbox.x1"],
                        caption["metadata"]["bbox.y1"],
                        caption["metadata"]["bbox.x2"],
                        caption["metadata"]["bbox.y2"],
                    )
                    if is_bbox_overlap(image_bbox, caption_bbox):
                        caption_text = caption["metadata"]["text_representation"]
                        break

                # Add caption to the image if available
                if caption_text:
                    new_height = img.height + 50
                    new_img = Image.new("RGB", (img.width, new_height), "white")
                    new_img.paste(img, (0, 0))
                    draw = ImageDraw.Draw(new_img)
                    font = ImageFont.load_default()
                    draw.text((10, img.height + 10), caption_text, fill="black", font=font)
                    img = new_img

                # Save the image
                img.save(os.path.join(output_dir, f"image_{idx + 1}.png"))
                print(f"Saved image with caption: image_{idx + 1}.png")
            except Exception as e:
                print(f"Failed to process image {idx + 1}: {e}")

    # Process Tables
    for idx, table in enumerate(tables):
        metadata = table["metadata"]
        try:
            # Retrieve table cells
            cells = metadata.get("table_cells", [])
            rows = {}
            for cell in cells:
                row_idx = cell["rows"][0]
                col_idx = cell["cols"][0]
                content = cell.get("content", "")
                if row_idx not in rows:
                    rows[row_idx] = {}
                rows[row_idx][col_idx] = content

            # Convert to DataFrame
            df = pd.DataFrame.from_dict(rows, orient="index").sort_index(axis=1)

            # Find associated caption
            caption_text = ""
            table_bbox = (
                metadata["bbox.x1"],
                metadata["bbox.y1"],
                metadata["bbox.x2"],
                metadata["bbox.y2"],
            )
            for caption in captions:
                caption_bbox = (
                    caption["metadata"]["bbox.x1"],
                    caption["metadata"]["bbox.y1"],
                    caption["metadata"]["bbox.x2"],
                    caption["metadata"]["bbox.y2"],
                )
                if is_bbox_overlap(table_bbox, caption_bbox):
                    caption_text = caption["metadata"]["text_representation"]
                    break

            # Save the table and caption
            table_path = os.path.join(output_dir, f"table_{idx + 1}.csv")
            df.to_csv(table_path, index=False)
            if caption_text:
                with open(table_path.replace(".csv", ".txt"), "w") as caption_file:
                    caption_file.write(caption_text)
            print(f"Saved table with caption: table_{idx + 1}.csv")
        except Exception as e:
            print(f"Failed to process table {idx + 1}: {e}")

# Utility function to check bounding box overlap
def is_bbox_overlap(bbox1, bbox2):
    return (
        bbox1[0] <= bbox2[2]
        and bbox1[2] >= bbox2[0]
        and bbox1[1] <= bbox2[3]
        and bbox1[3] >= bbox2[1]
    )

# Fetch and process Pinecone data
fetch_and_process_pinecone_data(index, output_dir)

Fetched data from Pinecone.


In [9]:
from pinecone import Pinecone
import base64
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import os

# Pinecone configuration
PINECONE_API_KEY = "pcsk_4P1CDJ_3Dx22sxGN8Vq2hS7TXe2MWgD2gp1QFe4z8Qnj4g86rcK3HSA9xiPXEjxpTXrLrt"
PINECONE_ENVIRONMENT = "us-east-1"
PINECONE_INDEX_NAME = "demo"

# Output directory
output_dir = "pinecone_extracted_data"
os.makedirs(output_dir, exist_ok=True)

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Check if the index exists
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    print(f"Index '{PINECONE_INDEX_NAME}' does not exist.")
    exit()

# Connect to the index
index = pc.Index(name=PINECONE_INDEX_NAME)

# Function to fetch and process Pinecone data
def fetch_and_process_pinecone_data(index, output_dir):
    # Query Pinecone for all items
    result = index.query(vector=[0] * 1536, top_k=1000, include_metadata=True)
    print("Query Results Retrieved")

    if not result.get("matches"):
        print("No matches found in Pinecone. Ensure the index contains data.")
        return

    # Separate images, tables, and captions
    images = [item for item in result["matches"] if item["metadata"].get("type") == "Image"]
    tables = [item for item in result["matches"] if item["metadata"].get("type") == "Table"]
    captions = [item for item in result["matches"] if item["metadata"].get("type") == "Caption"]

    print(f"Found {len(images)} images, {len(tables)} tables, and {len(captions)} captions.")

    # Process Images
    for idx, image in enumerate(images):
        metadata = image["metadata"]
        binary_data = metadata.get("binary_representation")
        if not binary_data:
            print(f"No binary data found for image {idx + 1}. Skipping.")
            continue
        try:
            print(f"Processing image {idx + 1}")
            img_data = base64.b64decode(binary_data)
            image_mode = metadata.get("properties.image_mode", "RGB")
            image_size = (int(metadata["properties.image_size.0"]), int(metadata["properties.image_size.1"]))
            img = Image.frombytes(image_mode, image_size, img_data)

            # Find associated caption
            caption_text = ""
            image_bbox = (
                metadata["bbox.x1"],
                metadata["bbox.y1"],
                metadata["bbox.x2"],
                metadata["bbox.y2"],
            )
            for caption in captions:
                caption_bbox = (
                    caption["metadata"]["bbox.x1"],
                    caption["metadata"]["bbox.y1"],
                    caption["metadata"]["bbox.x2"],
                    caption["metadata"]["bbox.y2"],
                )
                if is_bbox_overlap(image_bbox, caption_bbox):
                    caption_text = caption["metadata"]["text_representation"]
                    break

            # Add caption to the image if available
            if caption_text:
                new_height = img.height + 50
                new_img = Image.new("RGB", (img.width, new_height), "white")
                new_img.paste(img, (0, 0))
                draw = ImageDraw.Draw(new_img)
                font = ImageFont.load_default()
                draw.text((10, img.height + 10), caption_text, fill="black", font=font)
                img = new_img

            # Save the image
            img.save(os.path.join(output_dir, f"image_{idx + 1}.png"))
            print(f"Saved image with caption: image_{idx + 1}.png")
        except Exception as e:
            print(f"Failed to process image {idx + 1}: {e}")

    # Process Tables
    for idx, table in enumerate(tables):
        metadata = table["metadata"]
        try:
            print(f"Processing table {idx + 1}")
            # Retrieve table cells
            cells = metadata.get("table_cells", [])
            rows = {}
            for cell in cells:
                row_idx = cell["rows"][0]
                col_idx = cell["cols"][0]
                content = cell.get("content", "")
                if row_idx not in rows:
                    rows[row_idx] = {}
                rows[row_idx][col_idx] = content

            # Convert to DataFrame
            df = pd.DataFrame.from_dict(rows, orient="index").sort_index(axis=1)

            # Find associated caption
            caption_text = ""
            table_bbox = (
                metadata["bbox.x1"],
                metadata["bbox.y1"],
                metadata["bbox.x2"],
                metadata["bbox.y2"],
            )
            for caption in captions:
                caption_bbox = (
                    caption["metadata"]["bbox.x1"],
                    caption["metadata"]["bbox.y1"],
                    caption["metadata"]["bbox.x2"],
                    caption["metadata"]["bbox.y2"],
                )
                if is_bbox_overlap(table_bbox, caption_bbox):
                    caption_text = caption["metadata"]["text_representation"]
                    break

            # Save the table and caption
            table_path = os.path.join(output_dir, f"table_{idx + 1}.csv")
            df.to_csv(table_path, index=False)
            if caption_text:
                with open(table_path.replace(".csv", ".txt"), "w") as caption_file:
                    caption_file.write(caption_text)
            print(f"Saved table with caption: table_{idx + 1}.csv")
        except Exception as e:
            print(f"Failed to process table {idx + 1}: {e}")

# Utility function to check bounding box overlap
def is_bbox_overlap(bbox1, bbox2):
    return (
        bbox1[0] <= bbox2[2]
        and bbox1[2] >= bbox2[0]
        and bbox1[1] <= bbox2[3]
        and bbox1[3] >= bbox2[1]
    )

# Fetch and process Pinecone data
fetch_and_process_pinecone_data(index, output_dir)

Query Results Retrieved
Found 4 images, 0 tables, and 8 captions.
No binary data found for image 1. Skipping.
No binary data found for image 2. Skipping.
No binary data found for image 3. Skipping.
No binary data found for image 4. Skipping.
